In [0]:
SELECT * FROM hive_metastore.default.imdb_raw LIMIT 10;

In [0]:
CREATE OR REPLACE TABLE hive_metastore.default.dim_movie AS
SELECT DISTINCT
  Const AS movie_id,
  Title AS title,
  `Original Title` AS original_title,
  Year AS year,
  `Release Date` AS release_date,
  `IMDb Rating` AS imdb_rating,
  `Num Votes` AS num_votes,
  `Runtime (mins)` AS runtime,
  Genres AS genre,
  Directors AS director,
  URL AS url
FROM hive_metastore.default.imdb_raw
WHERE `Title Type` = 'Movie';


In [0]:
SELECT * FROM hive_metastore.default.dim_movie LIMIT 20;

In [0]:
CREATE OR REPLACE TEMP VIEW exploded_genres AS
SELECT 
  movie_id,  
  trim(genre_element) AS genre
FROM hive_metastore.default.dim_movie
LATERAL VIEW explode(split(genre, ",")) AS genre_element;

In [0]:
CREATE OR REPLACE TABLE hive_metastore.default.dim_genre AS
SELECT 
  ROW_NUMBER() OVER (ORDER BY genre) AS genre_id,
  TRIM(genre) AS genre
FROM (
  SELECT DISTINCT TRIM(genre) AS genre
  FROM exploded_genres
) t;
        

In [0]:
SELECT * FROM hive_metastore.default.dim_genre;

In [0]:
CREATE OR REPLACE TABLE hive_metastore.default.bridge_movie_genre AS
SELECT DISTINCT
  eg.movie_id,
  dg.genre_id
FROM exploded_genres eg
JOIN hive_metastore.default.dim_genre dg
  ON TRIM(eg.genre) = dg.genre;

In [0]:
SELECT 
  bmg.movie_id,
  COLLECT_LIST(dg.genre) AS genres
FROM hive_metastore.default.bridge_movie_genre bmg
JOIN hive_metastore.default.dim_genre dg
  ON bmg.genre_id = dg.genre_id
GROUP BY bmg.movie_id
ORDER BY bmg.movie_id;

In [0]:
CREATE OR REPLACE TEMP VIEW exploded_directors AS
SELECT
  movie_id,
  trim(director_element) AS director
FROM hive_metastore.default.dim_movie
LATERAL VIEW explode(split(director, ",")) AS director_element;

In [0]:
CREATE OR REPLACE TABLE hive_metastore.default.dim_director AS
SELECT
  ROW_NUMBER() OVER (ORDER BY director) AS director_id,
  TRIM(director) AS director
  FROM (
    SELECT DISTINCT TRIM(director) AS director
    FROM exploded_directors
  ) t;

In [0]:
CREATE OR REPLACE TABLE hive_metastore.default.bridge_movie_director AS
SELECT DISTINCT
  ed.movie_id,
  dd.director_id
FROM exploded_directors ed
JOIN hive_metastore.default.dim_director dd
  ON TRIM(ed.director) = dd.director;

In [0]:
SELECT *
FROM hive_metastore.default.bridge_movie_director
LIMIT 10;